# Notebook 04: Conception et Exécution du Workflow LangGraph

Ce notebook est dédié à l'exécution et à l'observation de notre workflow multi-agents "Cognitive Swarm", tel que défini dans `src/graph/main_workflow.py`. Nous allons soumettre une requête complexe et suivre le déroulement des opérations à travers les différents agents et outils.

**Prérequis :**
* Environnement configuré via `00_setup_environment.ipynb`.
* Base de données MongoDB peuplée (via `01_data_ingestion_and_embedding.ipynb` ou `scripts/run_ingestion.py`) pour que le `DocumentAnalysisAgent` ait des données à analyser.
* Les clés API (`OPENAI_API_KEY`, `WANDB_API_KEY` si le logger est utilisé, etc.) doivent être dans `.env`.
* Le `MongoDBSaver` (checkpointer) est activé par défaut dans la version actuelle de `main_workflow.py`. Assurez-vous que MongoDB est accessible.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio # Pour exécuter notre fonction de workflow asynchrone
import uuid   # Pour générer des thread_id uniques

# Ajout de la racine du projet au PYTHONPATH
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}.")

from config.settings import settings
from config.logging_config import setup_logging

# Importer la fonction d'exécution du workflow principal
from src.graph.main_workflow import run_cognitive_swarm_v2_1, graph_app_v2_1 # Importer aussi le graphe compilé

# Configurer le logging pour le notebook
LOG_LEVEL_NOTEBOOK = "INFO" # ou "DEBUG"
setup_logging(level=LOG_LEVEL_NOTEBOOK) 
logger = logging.getLogger("nb_04_workflow_execution")

# --- MODIFIÉ : Vérification des prérequis pour LLMs et Embeddings utilisés par le workflow ---
logger.info(f"--- Configuration Active pour le Workflow ---")
# Pour les LLMs génératifs utilisés par les agents du workflow
active_llm_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Fournisseur LLM génératif principal : '{active_llm_provider}'")
if active_llm_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
elif active_llm_provider == "huggingface_api" and not settings.HUGGINGFACE_API_KEY:
    logger.error(f"ERREUR : Le fournisseur LLM est 'huggingface_api', mais HUGGINGFACE_API_KEY n'est pas configurée.")
elif active_llm_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error(f"ERREUR : Le fournisseur LLM est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")

# Pour les embeddings (utilisés par RetrievalEngine via les outils)
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'Embedding (pour RAG) : '{active_embedding_provider}'")
if active_embedding_provider == "openai" and not settings.OPENAI_API_KEY:
    logger.error(f"ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée.")
elif active_embedding_provider == "ollama" and not settings.OLLAMA_BASE_URL:
    logger.error(f"ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée.")

# Pour MongoDB (checkpointer et RAG)
if not settings.MONGO_URI:
    logger.error("ERREUR : MONGO_URI non trouvé. Le checkpointer et le RetrievalEngine (RAG) échoueront.")
# --- FIN MODIFIÉ ---

# Petite fonction pour afficher l'état final de manière plus structurée
def pretty_print_final_state(final_state: dict):
    print("\n--- État Final Détaillé du Graphe ---")
    if not final_state:
        print("Aucun état final retourné.")
        return
        
    for key, value in final_state.items():
        if key == "messages":
            print(f"\n  {key.upper()}:")
            if isinstance(value, list):
                for i, msg in enumerate(value[-5:]): # Afficher les 5 derniers messages pour concision
                    msg_type = getattr(msg, 'type', 'UNKNOWN_MSG_TYPE').upper()
                    msg_name = getattr(msg, 'name', None)
                    msg_content_str = str(getattr(msg, 'content', 'N/A'))
                    display_name = f"{msg_type} ({msg_name})" if msg_name else msg_type
                    
                    print(f"    Message {len(value) - 5 + i +1 if len(value)>5 else i+1}: [{display_name}]")
                    if hasattr(msg, 'tool_calls') and msg.tool_calls:
                        print(f"      Contenu: {msg_content_str[:100]}... [Appels d'outils: {len(msg.tool_calls)}]")
                        for tc in msg.tool_calls:
                            print(f"        Tool Call ID: {tc.get('id')}, Name: {tc.get('name')}, Args: {tc.get('args')}")
                    elif msg_type == "TOOL": # ToolMessage
                        tool_call_id = getattr(msg, 'tool_call_id', 'N/A')
                        # Tenter de parser le contenu si c'est une chaîne JSON (pour les résultats d'outils structurés)
                        parsed_tool_content = None
                        if isinstance(msg_content_str, str):
                            try:
                                parsed_tool_content = json.loads(msg_content_str)
                            except json.JSONDecodeError:
                                pass # Laisser comme chaîne si ce n'est pas du JSON valide
                        
                        if parsed_tool_content and isinstance(parsed_tool_content, list) and parsed_tool_content:
                            print(f"      Tool Call ID: {tool_call_id} - Résultat Outil (Liste de {len(parsed_tool_content)} éléments):")
                            for item_idx, item_data in enumerate(parsed_tool_content[:2]): # Afficher les 2 premiers items
                                if isinstance(item_data, dict):
                                     print(f"        Item {item_idx+1}: { {k: str(v)[:70] + '...' if isinstance(v,str) and len(str(v)) > 70 else v for k,v in item_data.items()} }")
                                else:
                                     print(f"        Item {item_idx+1}: {str(item_data)[:100]}...")
                            if len(parsed_tool_content) > 2:
                                print("        ...")
                        else:
                            print(f"      Tool Call ID: {tool_call_id} - Contenu (Résultat Outil): {msg_content_str[:200]}...")
                    else:
                        print(f"      Contenu: {msg_content_str[:200]}...")
            else:
                print(f"    {str(value)[:500]}...")
        elif key == "research_plan" or key == "synthesis_output" or key == "document_analysis_summary":
            print(f"\n  {key.upper()}:\n{str(value)[:1000]}{'...' if value and len(str(value)) > 1000 else ''}\n")
        elif key == "user_query":
             print(f"\n  {key.upper()}: {value}")
        else: # Pour les autres clés de l'état
            print(f"  {key.upper()}: {str(value)[:500]}{'...' if value and len(str(value)) > 500 else ''}")
    print("------------------------------------")

### 1. Définition d'une Requête Utilisateur Complexe

Nous allons utiliser une requête qui nécessite potentiellement plusieurs étapes de la part de nos agents (planification, recherche, analyse, synthèse).

In [ ]:
user_query = "Analyze the latest advancements in reinforcement learning for robotic locomotion, focusing on how bipedal robots achieve stable gait. Include key challenges and future research directions based on recent ArXiv papers."
# user_query = "What are common methods for robot arm path planning based on recent ArXiv papers?"

logger.info(f"Requête utilisateur pour ce test : '{user_query}'")

### 2. Exécution du Workflow "Cognitive Swarm" (Premier Passage)

Nous exécutons ici la fonction `run_cognitive_swarm_v2_1` avec notre requête. Un nouvel ID de thread (`thread_id`) sera généré.
La sortie de `astream_events` dans `run_cognitive_swarm_v2_1` affichera le flux en temps réel (chunks de LLM, appels d'outils).
Le checkpointer MongoDB (activé par défaut dans `main_workflow.py`) sauvegardera l'état à chaque étape.

In [ ]:
# Générer un ID de thread unique pour cette exécution
test_thread_id = "nb_workflow_run_" + str(uuid.uuid4())

print(f"Lancement du workflow pour la requête avec thread_id: {test_thread_id}")
print("Les logs DEBUG du workflow (si activés) et les sorties des agents/outils via astream_events apparaîtront ci-dessous.")

# Exécuter le workflow
# Si vous exécutez ce notebook dans un environnement où une boucle d'événements asyncio est déjà en cours
# (par exemple, certains IDEs ou des versions de Jupyter), vous pourriez avoir besoin de `nest_asyncio`.
# import nest_asyncio
# nest_asyncio.apply()

final_state_run1 = None
if settings.OPENAI_API_KEY and settings.MONGO_URI: # Vérifications de base
    try:
        # Exécution de la fonction asynchrone
        final_state_run1 = asyncio.run(run_cognitive_swarm_v2_1(user_query, thread_id=test_thread_id))
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de asyncio.run(run_cognitive_swarm_v2_1): {e}", exc_info=True)
        print(f"ERREUR pendant l'exécution du workflow : {e}")
else:
    print("Clés API OpenAI ou MONGO_URI manquantes. Exécution du workflow annulée.")

# Afficher l'état final de manière plus structurée
if final_state_run1:
    pretty_print_final_state(final_state_run1)
else:
    print("L'exécution du workflow n'a pas retourné d'état final.")

### 3. Analyse des Sorties et du Comportement

Après l'exécution :
* Examinez les logs produits dans la console du notebook (si le niveau de log est DEBUG pour `main_workflow` ou les agents, vous verrez beaucoup de détails).
* Observez la sortie finale (`synthesis_output`) dans l'état final.
* Si vous avez accès à MongoDB (par exemple, via MongoDB Compass ou un autre client), vous pouvez inspecter la collection des checkpoints (par défaut `langgraph_checkpoints` dans la base `cognitive_swarm_db`). Vous devriez y trouver des documents correspondant au `thread_id` utilisé. Chaque document représente un état sauvegardé du graphe.

### 4. (Optionnel) Exécution d'une Requête de Suivi sur le Même Thread

Si le checkpointer a fonctionné, l'historique des messages et l'état du thread précédent sont sauvegardés. Envoyer une nouvelle requête avec le *même `thread_id`* permettra au système de potentiellement utiliser cet historique.

Notre workflow actuel est plutôt linéaire et ne gère pas explicitement les "questions de suivi" pour modifier un rapport existant. Une nouvelle invocation avec le même `thread_id` ajoutera à l'historique des messages et relancera le flux depuis le début, mais les agents verront l'historique complet.

Pour une vraie "reprise" d'un graphe interrompu, LangGraph le gère automatiquement si vous relancez avec la même configuration (thread_id).

In [ ]:
# # Décommentez pour tester un suivi.
# # Note: Cela relancera le flux avec l'historique accumulé.
#
# follow_up_query = "Can you provide more details on the challenges mentioned regarding sim-to-real transfer?"
# logger.info(f"Requête de suivi pour le thread {test_thread_id}: '{follow_up_query}'")
# print(f"\nLancement d'une requête de suivi sur le même thread_id: {test_thread_id}")

# final_state_run2 = None
# if settings.OPENAI_API_KEY and settings.MONGO_URI and test_thread_id: # S'assurer que test_thread_id est défini
#     try:
#         final_state_run2 = asyncio.run(run_cognitive_swarm_v2_1(follow_up_query, thread_id=test_thread_id))
#     except Exception as e:
#         logger.error(f"Erreur lors de l'exécution de la requête de suivi : {e}", exc_info=True)
#         print(f"ERREUR pendant l'exécution de la requête de suivi : {e}")
# else:
#     print("Conditions non remplies pour la requête de suivi (API Keys, MongoDB URI, ou thread_id manquant).")

# if final_state_run2:
#     pretty_print_final_state(final_state_run2)
# else:
#     print("L'exécution de la requête de suivi n'a pas retourné d'état final.")

### 5. Inspection des Checkpoints dans MongoDB (Conceptuel)

Si le `MongoDBSaver` est actif, vous pouvez vous connecter à votre instance MongoDB et examiner la collection `langgraph_checkpoints` (ou le nom que vous avez configuré dans `settings.py`). Vous y trouverez des documents JSON représentant les différents états sauvegardés pour chaque `thread_id`.

Chaque document de checkpoint contient typiquement :
* `thread_id`
* `thread_ts` (un timestamp identifiant ce snapshot spécifique de l'état)
* `checkpoint` (l'état sérialisé du graphe, incluant les valeurs des canaux comme `messages`)
* `metadata` (métadonnées associées au checkpoint)
* `parent_ts` (s'il y a un checkpoint parent)

Cela démontre la persistance et la capacité de reprise du workflow.

## Conclusion de la Démonstration du Workflow

Ce notebook a permis de lancer le workflow LangGraph complet et d'observer son exécution.
Les prochaines étapes pourraient inclure :
* Des tests avec des requêtes plus variées.
* L'analyse détaillée des checkpoints dans MongoDB.
* L'utilisation du script `scripts/run_evaluation.py` pour évaluer quantitativement les résultats.
* L'amélioration itérative de la logique de routage et des prompts des agents dans `src/graph/main_workflow.py` et `src/agents/agent_architectures.py`.